In [3]:
# Import packages

import numpy as np
import random
import matplotlib.pyplot as plt
from array2gif import write_gif
from tqdm import tqdm

In [22]:
# Create initial lattice to start

def create_random_lattice(rows, cols, prob = .5):

    n_trials = 1
    lattice = np.random.binomial(n_trials, prob, [rows, cols])
    lattice_idxs = np.where(lattice == 0)
    lattice[lattice_idxs] = -1

    return lattice

# Compute the probability (up to a normalizing constant) of the configuration

def compute_hamiltonian(lattice, J):

    num_rows = np.shape(lattice)[0]
    num_cols = np.shape(lattice)[1]
    hamiltonian = 0

    for ii_row in range(np.shape(lattice)[0]):

        for jj_col in range(np.shape(lattice)[1]):

            # Only count neighbors down and to the right to avoid double counting 
            # Periodic boundary conditions
            
            current_spin = lattice[ii_row, jj_col]
            hamiltonian += current_spin * lattice[(ii_row + 1) % num_rows, jj_col] + \
                current_spin * lattice[ii_row, (jj_col + 1) % num_cols]

    return -J * hamiltonian

def compute_unnormalized_prob(lattice, beta, J):

    return np.exp(-beta * compute_hamiltonian(lattice, J))

def propose_lattice(lattice):

    num_rows = np.shape(lattice)[0]
    num_cols = np.shape(lattice)[1]
    rnd_row = random.sample(list(range(num_rows)), 1)
    rnd_col = random.sample(list(range(num_cols)), 1)
    new_lattice = lattice
    new_lattice[rnd_row, rnd_col] = -1 * new_lattice[rnd_row, rnd_col]

    return new_lattice

def metropolis_sampling(lattice, num_iterations, J, beta):

    with tqdm(total = num_iterations) as pbar:

        sample = np.zeros([np.shape(lattice)[0], np.shape(lattice)[1], num_iterations])

        for ii_iteration in range(num_iterations):
            
            current_prob = compute_unnormalized_prob(lattice, beta, J)
            candidate_lattice = propose_lattice(lattice)
            candidate_prob = compute_unnormalized_prob(candidate_lattice, beta, J)
            acceptance_prob = np.min([1, candidate_prob / current_prob])

            if random.uniform(0, 1) < acceptance_prob:

                lattice = candidate_lattice

            sample[:, :, ii_iteration] = lattice
            pbar.update(1)
    
    return sample

# Write function that creates gif from the sample

def create_image(lattice):

    num_rows = np.shape(lattice)[0]
    num_cols = np.shape(lattice)[1]
    # Turn lattice into image with RGB triplet along third dimension
    image = np.zeros([num_rows, num_cols, 3])
    image[lattice == 1] = [0, 0, 0]
    image[lattice == -1] = [255, 255, 255]

    return image
        


In [55]:
num_rows = 200
num_cols = 200
initial_lattice = create_random_lattice(num_rows, num_cols)
num_iterations = 10000
J = 1
beta = 10
print('Metropolis sampling')
sample = metropolis_sampling(initial_lattice, num_iterations, J, beta)

  0%|          | 0/10000 [00:00<?, ?it/s]

Metropolis sampling


<ipython-input-22-df142bde1e01>:35: RuntimeWarning: overflow encountered in exp
  return np.exp(-beta * compute_hamiltonian(lattice, J))
<ipython-input-22-df142bde1e01>:59: RuntimeWarning: invalid value encountered in double_scalars
  acceptance_prob = np.min([1, candidate_prob / current_prob])
 88%|████████▊ | 8849/10000 [08:57<01:25, 13.40it/s]<ipython-input-22-df142bde1e01>:59: RuntimeWarning: divide by zero encountered in double_scalars
  acceptance_prob = np.min([1, candidate_prob / current_prob])
100%|██████████| 10000/10000 [10:24<00:00, 16.00it/s]


In [54]:
frequency = 5
dataset_for_gif = []

for ii_lattice in range(int(num_iterations / frequency)):

    dataset_for_gif.append(np.array(create_image(sample[:, :, ii_lattice * frequency])))

print('Writing samples to gif')
write_gif(dataset_for_gif, 'ising_model.gif', fps = 40)

Writing samples to gif
